In [1]:
#@title 1. 필수 라이브러리 설치 (버전 호환성 해결)
!pip install -q -U fastapi "uvicorn[standard]" pyngrok torch transformers peft "pandas==2.2.2" accelerate bitsandbytes huggingface_hub

In [2]:
#@title 2. 파일 업로드 및 Hugging Face 로그인
from google.colab import files
from huggingface_hub import login
import os

# --- 파일 업로드 ---
print("--- 📂 파일들을 업로드합니다 ---")
files_to_upload = ['styles.csv', 'gemma-fashion-tuner-intelligent.zip']
for filename in files_to_upload:
    if not os.path.exists(filename):
        print(f"\n'{filename}'을 업로드해주세요.")
        files.upload()
    else:
        print(f"'{filename}'이(가) 이미 존재합니다.")

# --- 모델 폴더 압축 해제 ---
print("\n모델 폴더 압축을 해제합니다...")
!unzip -q -o gemma-fashion-tuner-intelligent.zip # -o 옵션으로 덮어쓰기 허용
print("✅ 압축 해제 완료!")


# --- Hugging Face 로그인 ---
print("\n--- 🔐 Hugging Face 로그인이 필요합니다 ---")
# Gemma 베이스 모델 다운로드를 위해 Access Token을 입력해주세요.
login()
print("✅ 로그인 성공!")

--- 📂 파일들을 업로드합니다 ---
'styles.csv'이(가) 이미 존재합니다.
'gemma-fashion-tuner-intelligent.zip'이(가) 이미 존재합니다.

모델 폴더 압축을 해제합니다...
✅ 압축 해제 완료!

--- 🔐 Hugging Face 로그인이 필요합니다 ---


✅ 로그인 성공!


In [3]:
#@title 3. API 서버 코드 작성 (serve_intelligent.py) - 최종 완성 버전

%%writefile serve_intelligent.py

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import pandas as pd
from itertools import product
import time
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
import uvicorn

# --- 1. 모델 로딩 ---
print("✅ 지능형 모델 로딩을 시작합니다...")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ 사용할 장치: {device.upper()}")
base_model_id = "google/gemma-2b-it"
adapter_path = "/content/content/gemma-fashion-tuner-intelligent"
quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=quantization_config, device_map="auto")
model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.pad_token = tokenizer.eos_token
positive_token_id = tokenizer.encode("긍정적", add_special_tokens=False)[0]
negative_token_id = tokenizer.encode("부정적", add_special_tokens=False)[0]
print("✅ 모델 및 토크나이저 로딩 완료!")

# --- 2. 데이터 및 FastAPI 앱 설정 ---
styles_df = pd.read_csv("styles.csv", on_bad_lines='skip', dtype={'id': int}).set_index('id')
styles_info = styles_df.to_dict('index')
print("✅ styles.csv 로드 완료!")
app = FastAPI()

def get_item_description(item_id):
    if item_id not in styles_info: return ""
    info = styles_info[item_id]
    season = info.get('season', 'All')
    return f"{info.get('baseColour', '')} {info.get('articleType', '')} ({season})".strip()

class OutfitRequest(BaseModel):
    closet: List[int]; event: str; temperature: float; condition: str; gender: str

@app.post("/recommend_outfit")
def recommend_outfit(req: OutfitRequest):
    start_time = time.time()
    temp = req.temperature
    if temp <= 10: suitable_seasons = ['Winter', 'Fall']
    elif temp <= 18: suitable_seasons = ['Fall', 'Winter', 'Spring', 'All Season']
    else: suitable_seasons = ['Summer', 'Spring', 'All Season']

    closet_items = { 'Topwear': [], 'Bottomwear': [], 'Outerwear': [], 'Footwear': [], 'Full Body': [] }
    outerwear_types = ['Jackets', 'Blazers', 'Waistcoat', 'Coat', 'Shrug', 'Cardigan', 'Nehru Jackets', 'Rain Jacket', 'Sweaters']
    full_body_types = ['Dresses', 'Jumpsuit', 'Nightdress', 'Sarees', 'Kurta Sets', 'Night suits', 'Rompers', 'Bath Robe']

    for item_id in req.closet:
        if item_id in styles_info:
            info = styles_info[item_id]
            # ❗ 필터링 개선: 운동복(Sports)은 계절과 상관없이 포함
            if info.get('season') in suitable_seasons or info.get('season') == 'All Season' or info.get('usage') == 'Sports':
                sub_category = info.get('subCategory', ''); article_type = info.get('articleType', ''); master_category = info.get('masterCategory', '')
                if article_type in full_body_types: closet_items['Full Body'].append(item_id)
                elif article_type in outerwear_types: closet_items['Outerwear'].append(item_id)
                elif sub_category == 'Topwear': closet_items['Topwear'].append(item_id)
                elif sub_category == 'Bottomwear': closet_items['Bottomwear'].append(item_id)
                elif master_category == 'Footwear': closet_items['Footwear'].append(item_id)

    base_combinations = list(product(closet_items.get('Topwear', []), closet_items.get('Bottomwear', []), closet_items.get('Footwear', [])))
    outer_combinations = list(product(closet_items.get('Topwear', []), closet_items.get('Bottomwear', []), closet_items.get('Outerwear', []), closet_items.get('Footwear', [])))
    full_body_combinations = list(product(closet_items.get('Full Body', []), closet_items.get('Footwear', [])))
    all_combinations = [c for c in base_combinations + outer_combinations + full_body_combinations if c]

    if not all_combinations:
        return {"error": "현재 날씨와 상황에 맞는 유효한 조합을 옷장에서 찾을 수 없습니다."}

    prompts = [ f"상황: {req.gender}, {req.temperature}°C, {req.condition}, {req.event}\n옷: {', '.join([get_item_description(id) for id in combo])}\n이 조합은 적절한가요?\n결과:" for combo in all_combinations ]

    # --- ★ 추론 및 점수 계산 로직 수정 ---
    tokenizer.padding_side = "left"
    inputs = tokenizer(prompts, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    sequence_lengths = inputs['attention_mask'].sum(dim=1) - 1
    last_token_logits = outputs.logits[torch.arange(len(all_combinations)), sequence_lengths]

    # ❗ 안정적인 점수 계산: '긍정적'과 '부정적' 두 토큰에 대해서만 소프트맥스를 적용하여 확률 계산
    logits_for_scoring = last_token_logits[:, [negative_token_id, positive_token_id]]
    probs = torch.softmax(logits_for_scoring, dim=-1)
    all_scores = probs[:, 1].tolist() # '긍정적'일 확률 (0~1 사이의 값)

    best_score_index = all_scores.index(max(all_scores))
    best_score = all_scores[best_score_index]
    best_combo = all_combinations[best_score_index]
    best_combo_ids = [int(id_val) for id_val in best_combo if id_val is not None]
    outfit_str = ", ".join([get_item_description(id) for id in best_combo_ids])
    best_outfit_info = {"description": outfit_str, "ids": best_combo_ids}

    explanation = f"긍정적: {req.temperature}°C의 {req.condition} 날씨에 진행되는 {req.event}에 잘 어울리는 스타일입니다."

    end_time = time.time()
    return {"best_combination": best_outfit_info, "explanation": explanation, "best_score": best_score, "processing_time": end_time - start_time}

print("✅ 'serve_intelligent.py' 파일이 성공적으로 생성되었습니다.")

Overwriting serve_intelligent.py


In [4]:
# 폴더의 실제 내부 구조를 확인합니다.
!ls -lR /content/content/gemma-fashion-tuner-intelligent

/content/content/gemma-fashion-tuner-intelligent:
total 44996
-rw-r--r-- 1 root root      880 Aug 26 13:50 adapter_config.json
-rw-r--r-- 1 root root  7391688 Aug 26 13:50 adapter_model.safetensors
-rw-r--r-- 1 root root      591 Aug 26 13:50 chat_template.jinja
drwxr-xr-x 2 root root     4096 Aug 27 02:14 checkpoint-500
-rw-r--r-- 1 root root     5190 Aug 26 13:50 README.md
drwxr-xr-x 4 root root     4096 Aug 26 08:53 runs
-rw-r--r-- 1 root root      522 Aug 26 13:50 special_tokens_map.json
-rw-r--r-- 1 root root    40021 Aug 26 13:50 tokenizer_config.json
-rw-r--r-- 1 root root 34356305 Aug 26 13:50 tokenizer.json
-rw-r--r-- 1 root root  4241003 Aug 26 13:50 tokenizer.model
-rw-r--r-- 1 root root     5777 Aug 26 13:50 training_args.bin

/content/content/gemma-fashion-tuner-intelligent/checkpoint-500:
total 59540
-rw-r--r-- 1 root root      880 Aug 26 08:49 adapter_config.json
-rw-r--r-- 1 root root  7391688 Aug 26 08:49 adapter_model.safetensors
-rw-r--r-- 1 root root      591 Aug 26

In [ ]:
#@title 4. 서버 실행 및 테스트
import threading
import uvicorn
import time
import requests
from pyngrok import ngrok

# --- 서버를 백그라운드 스레드에서 실행 ---
def run_app():
  # serve_intelligent.py를 uvicorn으로 실행
  uvicorn.run("serve_intelligent:app", host="0.0.0.0", port=8000, log_level="info")

thread = threading.Thread(target=run_app)
thread.start()
print("✅ FastAPI 서버가 백그라운드에서 실행을 시작했습니다...")

# --- 서버가 켜질 때까지 대기 (Health Check) ---
print("⏳ 서버가 모델을 로딩하고 준비될 때까지 대기 중입니다... (최대 10분)")
start_time = time.time()
server_ready = False
while time.time() - start_time < 600: # 최대 10분 대기
    try:
        response = requests.get("http://localhost:8000/", timeout=5)
        if response.status_code == 404:
            print("✅ 서버가 응답하기 시작했습니다!")
            server_ready = True
            break
    except requests.exceptions.ConnectionError:
        time.sleep(5)
        print("...")

if not server_ready:
    print("❌ 10분이 지나도 서버가 준비되지 않았습니다. 런타임을 재시작하고 다시 시도해주세요.")
else:
    # --- ngrok으로 외부 접속 터널 생성 ---
    # https://dashboard.ngrok.com/get-started/your-authtoken 에서 토큰을 복사하여 붙여넣으세요.
    NGROK_TOKEN = "31HHkLEWGt90qDRoAWd6BqiGL9K_5fJsXN7LgijbtcAVwkwAS" # 🚨
    ngrok.set_auth_token(NGROK_TOKEN)
    ngrok.kill()
    public_url = ngrok.connect(8000)
    print(f"\n🎉 서버가 성공적으로 생성되었습니다. API 주소: {public_url}")
    print("이제 이 주소를 test_api.py에 넣고 로컬에서 테스트를 실행하세요.")

thread.join()

✅ FastAPI 서버가 백그라운드에서 실행을 시작했습니다...
⏳ 서버가 모델을 로딩하고 준비될 때까지 대기 중입니다... (최대 10분)
...
...
...
✅ 지능형 모델 로딩을 시작합니다...
✅ 사용할 장치: CUDA


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...
✅ 모델 및 토크나이저 로딩 완료!


INFO:     Started server process [47286]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


✅ styles.csv 로드 완료!
✅ 'serve_intelligent.py' 파일이 성공적으로 생성되었습니다.
...
INFO:     127.0.0.1:49538 - "GET / HTTP/1.1" 404 Not Found
✅ 서버가 응답하기 시작했습니다!

🎉 서버가 성공적으로 생성되었습니다. API 주소: NgrokTunnel: "https://21ac6154f3fa.ngrok-free.app" -> "http://localhost:8000"
이제 이 주소를 test_api.py에 넣고 로컬에서 테스트를 실행하세요.
INFO:     39.113.75.42:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     39.113.75.42:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     39.113.75.42:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     39.113.75.42:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     39.113.75.42:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     39.113.75.42:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
